<a href="https://colab.research.google.com/github/himanshu530/Complete-Deep-Learning/blob/master/cnn_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CNN Model + Optimization

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=5c932f0bbf50bb69016ea3a4a3fa348607ec5dd4fc5570b90a2d1da318b97542
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=290fda682db54b772355c2bd501927d5a8ce9e37696557af6ac2982f9de2646c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import  keras
import numpy as np

In [4]:
print(tf.__version__)

2.3.0


In [5]:
 fashion_mnist = keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [10]:
#Changing from grayscale

train_images = train_images/255.0
test_images = test_images/255.0

In [11]:
#Important step is reshaping the data
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [12]:
train_images[0].shape

(28, 28)

In [13]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [14]:
train_images[0].shape

(28, 28, 1)

In [17]:
#Function to create a model by tuning the hyperparameters
def build_model(hp):
  model = Keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=32, max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter',min_value=32,max_value=64,step = 16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units',min_value=32,max_value=128,step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_entropy',
                metrics=['accuracy'])
  return model

In [33]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
#Running random_search for best params
tuner_search = RandomSearch(build_model,objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,intial_epoch=3)